In [1]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import seaborn as sns
import seml.database as db_utils

import jkutils.mpl_latex_template as mplt

import os 
os.environ["PATH"] += os.pathsep + '/nfs/homedirs/schuchaj/texlive/2020/bin/x86_64-linux'

/nfs/homedirs/schuchaj/miniconda3/envs/sparse/lib/python3.7/_collections_abc.py:841: MatplotlibDeprecationWarning: Support for setting the 'text.latex.preamble' or 'pgf.preamble' rcParam to a list of strings is deprecated since 3.3 and will be removed two minor releases later; set it to a single string instead.
  self[key] = other[key]


In [2]:
collection = 'cert_collective'
collection_train = 'train_collective'

jk_config = {
    'username': 'schuchaj',
    'password': 'LBGqsHEY',
    'host': 'fs.kdd.in.tum.de',
    'port': 27017,
    'db_name': 'schuchaj'
}

col = db_utils.get_collection(collection, mongodb_config=jk_config)
col_train = db_utils.get_collection(collection_train, mongodb_config=jk_config)

In [72]:
def find_items(dataset, model, max_rad, pf_minus_adj, pf_plus_att, pf_minus_att, seed=None, kl_loss_weight=None, top_k=None):
    
    db_filter = {
        'config.restrictions.dataset' : dataset,
        'config.restrictions.model' : model,
        'config.max_rad' : max_rad,
        'config.pf_plus_adj': 0,
        'config.pf_minus_adj': pf_minus_adj,
        'config.pf_plus_att': pf_plus_att,
        'config.pf_minus_att': pf_minus_att,
        'config.local_budget_descriptor': None,
        'config.restrictions.kl_loss_weight': kl_loss_weight,
        'config.restrictions.top_k': None
    }
    if model == 'APPNP':
        db_filter['config.restrictions.top_k'] = 32
    if seed is not None:
        db_filter['config.seed'] = seed
    if col.count_documents(db_filter) == 0:
        raise ValueError('No matches!')

    exps = col.find(db_filter)
    
    ret = []
    
    for exp in exps:
        config = exp['config']
        cert_file_dir = exp['result']['cert_file']
        try:
            cert = torch.load(cert_file_dir)
        except:
            cert = torch.load(f'/nfs/students/schuchaj/sparse_smoothing_results/cert_collective/{cert_file_dir.split("/")[-1]}')

        ret.append((exp, config, cert))
            
    return ret

In [73]:
def find_items_train(dataset, model, pf_minus_adj, pf_plus_att, pf_minus_att, seed=None, kl_loss_weight=None, top_k=None):
    db_filter = {
        'config.dataset' : dataset,
        'config.model': model,
        'config.pf_plus_adj': 0,
        'config.pf_minus_adj': pf_minus_adj,
        'config.pf_plus_att': pf_plus_att,
        'config.pf_minus_att': pf_minus_att,
        'config.kl_loss_weight': kl_loss_weight,
        'config.top_k': top_k
    }

    if seed is not None:
        db_filter['config.seed'] = seed

    if col_train.count_documents(db_filter) == 0:
        raise ValueError('No matches!')

    exps = col_train.find(db_filter)
    
    ret = []
    
    for exp in exps:
        config = exp['config']
        
        run_id = config['overwrite']
        db_collection = config['db_collection']
        save_dir = config['save_dir']

        try:
            cert = torch.load(f'{save_dir}/{db_collection}_{run_id}', map_location=torch.device('cpu'))
        except:
            cert = torch.load(f'/nfs/students/schuchaj/sparse_smoothing_results/train_collective/{db_collection}_{run_id}', map_location=torch.device('cpu'))

        ret.append((exp, config, cert))
            
    return ret

In [74]:
def grid_from_exp(exp, collective=True):
    cert = exp[2]
    if collective:
        return cert['collective_grid']
    else:
        idx_test = cert['idx_test']
        return ((cert['grid_base'][idx_test] > 0.5).sum(axis=0))

In [75]:
def plot(grids, color, linestyle, x=None, label=None, plot_std=False, std_alpha=0.15, step=False, pad_last=False):
    mean = np.mean(grids, axis=0)
    
    if pad_last:
        mean = np.hstack([mean, [mean[-1]]])

    if x is None:
        x = np.arange(len(mean))
    
    if step:
        plt.step(x, mean, c=color, linestyle=linestyle, label=label, where='post', clip_on=False, zorder=3)
    else:
        plt.plot(x, mean, c=color, linestyle=linestyle, label=label, clip_on=False, zorder=3)
    
    if plot_std:
        std = np.std(grids, axis=0)
        if pad_last:
            std = np.hstack([std, [std[-1]]])

        if step:
            plt.fill_between(x, mean-std, mean+std, color=color, alpha=std_alpha, step='post')
        else:
            plt.fill_between(x, mean-std, mean+std, color=color, alpha=std_alpha)

In [76]:
# Styling
sns.set()
pal = sns.color_palette('colorblind', 4)

In [80]:
mplt.newfig(width=0.49, textwidth_pt=397.48499)



exps_collective_gat = find_items('cora_ml', 'GAT', [0, 0, -1], 0, 0.002, 0.6)
grids_gat = np.vstack([grid_from_exp(exp)[0, 0] / grid_from_exp(exp)[0, 0, 0] for exp in exps_collective_gat])
plot(grids_gat, pal[1], '-', label='GAT', plot_std=True, step=True, pad_last=True)

exps_collective_gcn = find_items('cora_ml', 'GCN', [0, 0, -1], 0, 0.002, 0.6)
grids_gcn = np.vstack([grid_from_exp(exp)[0, 0] / grid_from_exp(exp)[0, 0, 0] for exp in exps_collective_gcn])
plot(grids_gcn, pal[0], '-', label='GCN', plot_std=True, step=True, pad_last=True)

exps_collective_rgcn = find_items('cora_ml', 'RGCN', [0, 0, -1], 0, 0.002, 0.6, kl_loss_weight=0.00000)
grids_rgcn = np.vstack([grid_from_exp(exp)[0, 0] / grid_from_exp(exp)[0, 0, 0] for exp in exps_collective_rgcn])
plot(grids_rgcn, pal[2], '-', label='RGCN', plot_std=True, step=True, pad_last=True)

exps_collective_appnp = find_items('cora_ml', 'APPNP', [0, 0, -1], 0, 0.002, 0.6, top_k=64)
grids_appnp = np.vstack([grid_from_exp(exp)[0, 0] / grid_from_exp(exp)[0, 0, 0] for exp in exps_collective_appnp])
plot(grids_appnp, pal[3], '-', label='APPNP', plot_std=True, step=True, pad_last=True)



exps_train_gat = find_items_train('cora_ml', 'GAT', 0, 0.002, 0.6)
grids_train_gat = np.vstack([grid_from_exp(exp, collective=False)[0] / grid_from_exp(exp, collective=False)[0, 0] for exp in exps_train_gat])
plot(grids_train_gat, pal[1], (0, (6, 2)), plot_std=True, step=True)

exps_train_gcn = find_items_train('cora_ml', 'GCN', 0, 0.002, 0.6)
grids_train_gcn = np.vstack([grid_from_exp(exp, collective=False)[0] / grid_from_exp(exp, collective=False)[0, 0] for exp in exps_train_gcn])
plot(grids_train_gcn, pal[0], (0, (6, 2)), plot_std=True, step=True)


exps_train_rgcn = find_items_train('cora_ml', 'RGCN', 0, 0.002, 0.6, kl_loss_weight=0.00000)
grids_train_rgcn = np.vstack([grid_from_exp(exp, collective=False)[0] / grid_from_exp(exp, collective=False)[0, 0] for exp in exps_train_rgcn])
plot(grids_train_rgcn, pal[2], (0, (6, 2)), plot_std=True, step=True)

exps_train_appnp = find_items_train('cora_ml', 'APPNP', 0, 0.002, 0.6, kl_loss_weight=0.0005, top_k=64)
grids_train_appnp = np.vstack([grid_from_exp(exp, collective=False)[0] / grid_from_exp(exp, collective=False)[0, 0] for exp in exps_train_appnp])
plot(grids_train_appnp, pal[3], (0, (6, 2)), plot_std=True, step=True)


plt.legend()
plt.xlabel('Attribute deletions')
plt.ylabel('Certified ratio')
plt.ylim(0, 1)
plt.xlim(0, 22)

plt.tight_layout()

mplt.savefig(f'/nfs/homedirs/schuchaj/plotting/figures/experiments/models/small_radii', format='pgf', preview='png', dpi=200, tight={'pad': 0.5})

/nfs/homedirs/schuchaj/miniconda3/envs/sparse/lib/python3.7/_collections_abc.py:841: MatplotlibDeprecationWarning: Support for setting the 'text.latex.preamble' or 'pgf.preamble' rcParam to a list of strings is deprecated since 3.3 and will be removed two minor releases later; set it to a single string instead.
  self[key] = other[key]


Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello
Hello


In [ ]:
print((grids_gcn / grids_train_gcn)[:, :-1].mean())
print((grids_gat / grids_train_gat)[:, :-1].mean())
print((grids_rgcn / grids_train_rgcn)[:, :-1].mean())

In [ ]:
mplt.newfig(width=0.49, textwidth_pt=397.48499)

exps_collective_gcn = find_items('cora_ml', 'GCN', [-1, 0, 0], 0.4, 0.00, 0.)
grids_gcn = np.vstack([grid_from_exp(exp)[:, 0, 0] / grid_from_exp(exp)[0, 0, 0] for exp in exps_collective_gcn])
plot(grids_gcn, pal[0], '-', label='GCN', plot_std=True, step=True, pad_last=True)

exps_collective_gat = find_items('cora_ml', 'GAT', [-1, 0, 0], 0.4, 0.00, 0)
grids_gat = np.vstack([grid_from_exp(exp)[:, 0, 0] / grid_from_exp(exp)[0, 0, 0] for exp in exps_collective_gat])
plot(grids_gat, pal[1], '-', label='GAT', plot_std=True, step=True, pad_last=True)

exps_collective_rgcn = find_items('cora_ml', 'RGCN', [-1, 0, 0], 0.4, 0.00, 0., kl_loss_weight=0.0000)
grids_rgcn = np.vstack([grid_from_exp(exp)[:, 0, 0] / grid_from_exp(exp)[0, 0, 0] for exp in exps_collective_rgcn])
plot(grids_rgcn, pal[2], '-', label='RGCN', plot_std=True, step=True, pad_last=True)


exps_train_gcn = find_items_train('cora_ml', 'GCN', 0.4, 0.00, 0.)
grids_train_gcn = np.vstack([grid_from_exp(exp, collective=False)[0] / grid_from_exp(exp, collective=False)[0, 0] for exp in exps_train_gcn])
plot(grids_train_gcn, pal[0], (0, (6, 2)), plot_std=True, step=True)

exps_train_gat = find_items_train('cora_ml', 'GAT', 0.4, 0.00, 0.)
grids_train_gat = np.vstack([grid_from_exp(exp, collective=False)[0] / grid_from_exp(exp, collective=False)[0, 0] for exp in exps_train_gat])
plot(grids_train_gat, pal[1], (0, (6, 2)), plot_std=True, step=True)

exps_train_rgcn = find_items_train('cora_ml', 'RGCN', 0.4, 0.00, 0., kl_loss_weight=0.0000)
grids_train_rgcn = np.vstack([grid_from_exp(exp, collective=False)[0] / grid_from_exp(exp, collective=False)[0, 0] for exp in exps_train_rgcn])
plot(grids_train_rgcn, pal[2], (0, (6, 2)), plot_std=True, step=True)


plt.legend()
plt.xlabel('Edge deletions')
plt.ylabel('Certified ratio')

plt.ylim(0, 1)
plt.xlim(0, 13)

plt.tight_layout()

mplt.savefig(f'/nfs/homedirs/schuchaj/plotting/figures/experiments/models/small_radii_edges', format='pgf', preview='png', dpi=200, tight={'pad': 0.5})

In [11]:
print((grids_gcn / grids_train_gcn)[:, :-1].mean())
print((grids_gat / grids_train_gat)[:, :-1].mean())
print((grids_rgcn / grids_train_rgcn)[:, :-1].mean())

1.3994232630115746
1.4520792584686133
1.3967460411471158


/nfs/homedirs/schuchaj/miniconda3/envs/sparse/lib/python3.7/site-packages/ipykernel_launcher.py:1: RuntimeWarning: divide by zero encountered in true_divide
  """Entry point for launching an IPython kernel.
/nfs/homedirs/schuchaj/miniconda3/envs/sparse/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in true_divide
  
/nfs/homedirs/schuchaj/miniconda3/envs/sparse/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until
